In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    ##for filename in filenames:
        #print(os.path.join(dirname, filename))
#from matplotlib.image import imread  

from cv2 import imread
import matplotlib.pyplot as plt

train_path = '../input/state-farm-distracted-driver-detection/imgs/train/'
test_path = '..//input/state-farm-distracted-driver-detection//imgs//test//'
#test_path_2 = '..//input/state-farm-distracted-driver-detection//imgs//'
#print(os.listdir(train_path))

train_data=[]
test_data=[]
classes = [folder for folder in os.listdir(train_path)]

# explanation for each of the classes
class_dict = {
    'c0': 'hands on the wheel',
    'c1': 'mobile in right hand',
    'c2': 'talking on the phone with right hand',
    'c3': "mobile in left hand",
    'c4': 'talking on the phone with left hand',
    'c5': 'touching at the dash',
    'c6': 'drinking',
    'c7': 'reaching behind',
    'c8': 'touching the head',
    'c9': 'looking to the side'
}



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## checking the dimensions of images in test data set so that we can resize the images accordingly.
import random
dim1 =[]
dim2=[]
image_list = os.listdir(test_path)
image_list = random.sample(image_list,500)
for image in image_list:
    path = os.path.join(test_path,image)
    #print(path)
    d1,d2,channels = imread(path).shape
    dim1.append(d1)
    dim2.append(d2)
    


In [ ]:
print(np.mean(dim1))
print(np.mean(dim2))

# we can see that the dimension of each image is (480,640,3) from a sample of 500 randomly selected test images.

In [ ]:
import cv2
for item in classes:
    path = os.path.join(train_path,item)
    #count=0
    for image in os.listdir(path):
        image_array = imread(os.path.join(path,image))#,cv2.IMREAD_COLOR)
        count+=1
        #print(image_array.shape)
        print(item)
        print(class_dict[item])
        plt.imshow(image_array,cmap='gray')
        plt.show()
        print('\n')
        break
        

In [ ]:
from tensorflow .keras.preprocessing.image import ImageDataGenerator

In [ ]:
#         train_datagen = ImageDataGenerator(
#         rescale=1./255,
#         #rotation_range=30,
#         shear_range=0.1,
#         width_shift_range = 0.2,
#         height_shift_range = 0.2,
#         fill_mode='nearest',
#         zoom_range=0.2,
#         #brightness_range= [0.1,1]
#         #validation_split=0.2
#         )

In [ ]:
#         train_generator = train_datagen.flow_from_directory(
#         train_path,
#         target_size=(224, 224),
#         batch_size=32,
#         class_mode='categorical',
#         #subset='training'
#         )


In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        #featurewise_std_normalization=True,
        #featurewise_center = True,
        shear_range=0.1,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        fill_mode='nearest',
        #zoom_range=0.2,
        #horizontal_flip=True,
        #vertical_flip=True,
        #brightness_range= [0.1,1],
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training')

test_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation',
        shuffle=True)



In [ ]:
## VERIFYING THE IMAGE AND THE LABELS

# batch =  train_generator.next()
# #print(batch.shape)
# for i ,j in zip(batch[0],batch[1]):
#     plt.imshow(i)
#     plt.show()
#     print(class_dict['c'+str(np.argmax(j))])
#     print('\n')
    
    

In [ ]:
#next(train_generator)

In [ ]:
#train_generator.next()[0].shape # x is scaled we can see
#train_generator.next()[1]# y is one hot encoded as well

In [ ]:
## Lets Create Model Now

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
import tensorflow as tf


In [ ]:
model = Sequential()

In [ ]:
## convolution layer 1 and max pool 1


model.add(Conv2D(64,(2,2),input_shape = (224,224,3), activation='relu',padding='valid'))
model.add(MaxPool2D(pool_size=(3,3)))

## convolution layer 2 and max pool 2

model.add(Conv2D(128,(2,2), activation='relu',padding='valid'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(256,(2,2), activation='relu',padding='valid'))
model.add(MaxPool2D(pool_size=(2,2)))
## flattening the image before going to hidden layer
model.add(Flatten())



## Dropout layer 1 and Hidden layer 1

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

## Hidden layer 2

model.add(Dense(256,activation ='relu'))
model.add(Dropout(0.2))


## output layer

model.add(Dense(10,activation='softmax'))# softmax because of multiclass classification


#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




In [ ]:
## Compiling the model now

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
Early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
#model.fit_generator(train_generator, epochs=10,   callbacks=[Early_stop],use_multiprocessing=True)
   
    

In [ ]:
model.fit_generator(train_generator, epochs=15,  validation_data=test_generator, callbacks=[Early_stop],use_multiprocessing=True)
   
    

In [ ]:
model.metrics_names

In [ ]:
predictions = model.predict_generator(test_generator, use_multiprocessing=True)

In [ ]:
## checking the predictions on validation generator and matching with the image .
import cv2
# for i in range(50):
batch = test_generator.next()
for i,j in zip(batch[0],batch[1]):
    plt.imshow(i)
    
    plt.show()
    #plt.show()
    print('Actual: ', class_dict['c'+str(np.argmax(j))])
    #print('\n')
    #print('\n')
    #break

    #image =cv2.resize(image,(224,224))
    image=i.reshape(1,224,224,3)
    print('predicted: ', class_dict['c'+ str(np.argmax(model.predict(image)))])
    print('\n')

In [ ]:
model.evaluate_generator(test_generator,use_multiprocessing=True)

In [ ]:
## we can see that we are almost getting 97 percent accuracy on the validation data as well.

In [ ]:
# import cv2
# for image in os.listdir(test_path)[4:]:
#     path = os.path.join(test_path,image)
#     print(path)
#     image = cv2.imread(path)
#     plt.imshow(image)
#     break
    

In [ ]:
# ## checking the output on test folder manually though no label is there in the images in test folder.
# import cv2
# for image in os.listdir(test_path)[10:40]:
#     path = os.path.join(test_path,image)
#     #print(path)
#     image = cv2.imread(path)
#     plt.imshow(image)
#     plt.show()
#     #print(image.shape)
#     #print('\n')
#     #break

#     image =cv2.resize(image,(224,224))
#     image=image.reshape(1,224,224,3)
#     print('predicted : ' , class_dict['c'+ str(np.argmax(model.predict(image)))])
#     print('\n')